## Figures for Network model of Latent Recurrent Discharge (LRD)


### Some imports and settings


In [ ]:
SAVE_FIGS = True  # see save_args above for configuration
DOWNLOAD_CACHE = False  # if you want the cache files, set to True instead of running simulations again

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings

import brian2.numpy_ as np
import matplotlib.pyplot as plt

from settings import COLOR, G_GABA_LIST, TAU_KCC2_LIST, constants, logging, time_unit

# start from defaults before loading stylesheet
plt.style.use("default")
plt.style.use("style/article.mplstyle")

# suppress warnings
np.seterr(all="ignore")
warnings.filterwarnings("ignore")

logger = logging.getLogger("lrd")

save_args = dict(use_args=False, close=False, file_formats=["svg", "jpg"])

In [ ]:
if DOWNLOAD_CACHE:
    %pip install git+https://github.com/ChrisCurrin/dropbox-downloader.git
    import dropbox

    dropbox.downloadFile("https://www.dropbox.com/sh/noqzojddijvvs2a/AAD8ppgjSyi7QIcoGu0e3Ylba?dl=0", ".", unzip=True, retain_zip=False)


# Explain figure


In [ ]:
from scripts.lrdfigure_explain import Explain

np.random.seed(1337)

explain = Explain()

explain.run(
    # mv_step=4,
    # time_per_value=40,
    # egaba=[-78, -34],
    mv_step=2,
    time_per_value=60,
    egaba=[-74, -40],
)

explain.plot()
if SAVE_FIGS:
    explain.save_figure(**save_args)
plt.show()

# Alter conductances

1. Emulate picrotoxin (pink) by **reducing** $g_{GABA_{max}}$.
2. Emulate benzodiazepines (green) by **increasing** $g_{GABA_{max}}$.

We can see the effect of benzodiazepines depends on $E_{GABA}$.


In [ ]:
from scripts.lrdfigure_drugs import Drugs

drugs = Drugs(benzo_strengths=(0, 0.25, 0.5, 1, 2, 4, 8), egabas=[-74, -60, -46])
drugs.run()
drugs.plot(drugs_to_plot=[0.25, 4])
if SAVE_FIGS:
    drugs.save_figure(**save_args)

## Start having dynamic chloride $Cl^-$

We compared networks with either hyperpolarised ($\approx E_{GABA} = -74 mV$) or depolarised ($ \approx E\_{GABA} = -46 mV$) EGABA.

The starting $E_{GABA}$ seems to have neglible, if any effect.

We further consider the effect of GABA conductance $g_{GABA_{max}}$ (brightness, rows) along with different $\tau_{KCC2}$ values (hue, traces).


In [ ]:
from scripts.lrdfigure_chloride import Chloride

cl = Chloride(
    tau_KCC2s=TAU_KCC2_LIST[::2][:-1],
    E_Cl_0s=(-60, -88),
    g_GABAs=(50, 25, 100),
    seeds=(None, 1038, 1337, 1111, 1010, 1011, 1101, 1110, 11110, 111100),
)
cl.run(duration=600)
cl.plot(timeit=True, colorbar=False)
if SAVE_FIGS:
    cl.save_figure(**save_args)
del cl

### Supplementary work: the effect of neuron size ($L$) and chloride extrusion ($\tau_{KCC2}$) on the number of bursts in a network


In [ ]:
from scripts.lrdfigure_chloride_length import ChlorideLength

cl_length = ChlorideLength(
    g_GABAs=(50, 25, 100),
    lengths=(
        15,
        # 10,
        7.5,
        # 6.25,
        5,
    ),
    seeds=(
        None,
        1038,
        1337,
        1111,
        1234,
    ),
)

cl_length.run(duration=600)
cl_length.plot(timeit=True, colorbar=False)
if SAVE_FIGS:
    cl_length.save_figure(**save_args)

# Explore time constants of "Excitatory" and "Inhibitory" KCC2

$\tau_{KCC2_{exc}}$ = [15, 21, 30, 42, 60, 84, 120, 169, 240]

$\tau_{KCC2_{inh}}$ = [15, 21, 30, 42, 60, 84, 120, 169, 240]


In [ ]:
from scripts.lrdfigure_tau import Tau

tau = Tau(
    tau_KCC2_E_list=TAU_KCC2_LIST,
    tau_KCC2_I_list=TAU_KCC2_LIST,
    g_GABA_list=G_GABA_LIST + [37, 75, 150],
    seeds=(
        None,
        1038,
        1337,
        1111,
        1010,
    ),
)

tau.run(duration=600, nrn_idx_i=[0, 1, 2, 3])
(df_long, df_bursts_bins, df_taus, df_num_bursts) = tau.process_data()
tau.plot(
    plot_ggaba=[25, 100],
    df_num_bursts=df_num_bursts,
    vmax=df_num_bursts[df_num_bursts["g_GABA"].isin([25, 150])][
        "Number of bursts"
    ].max(),
)
if SAVE_FIGS:
    tau.save_figure(**save_args)

In [ ]:
tau.plot_old(plot_g_GABA_list=[25, 100])

## $g_{GABA_{max}}$ vs $\tau_{KCC2}$ with $E_{GABA}$

First, plot number of bursts as a function of $g_{GABA_{max}}$ as we directly vary $E_{GABA}$ (static $Cl^-$).

Next, plot number of bursts and $E_{GABA}$ as a function of $g_{GABA_{max}}$ and $\tau_{KCC2}$.

Finally, plot the relationship between the pre-burst GABAergic current ($I_{GABA}$) and the number of bursts. The $g_{GABA_{max}}$ and $\tau_{KCC2}$ are displayed for reference as color and marker size, respectively. Squares are from static simulations.


In [ ]:
from scripts.lrdfigure_ggaba_e import Gve

# add some more lower values for tau
tau_KCC2_list = TAU_KCC2_LIST

ratio = tau_KCC2_list[1] / tau_KCC2_list[0]
tau_KCC2_list = [np.round(tau_KCC2_list[0] / ratio, 1)] + tau_KCC2_list
tau_KCC2_list = [np.round(tau_KCC2_list[0] / ratio, 1)] + tau_KCC2_list
tau_KCC2_list = [np.round(tau_KCC2_list[0] / ratio, 1)] + tau_KCC2_list
tau_KCC2_list = [np.round(tau_KCC2_list[0] / ratio, 1)] + tau_KCC2_list


gve = Gve(
    seeds=(None, 1234, 5678, 1426987, 86751, 1010, 876, 12576, 9681, 814265),
    gGABAsvEGABA=sorted(
        set(
            np.append(
                np.round(np.arange(0, 100.0001, 10), 0),
                np.geomspace(10, 1000, 11).round(0),
            )
        )
    ),
    gGABAs=np.geomspace(10, 1000, 11).round(0),
    tau_KCC2s=tau_KCC2_list,
)
gve.run()
gve.process()
import seaborn as sns

fig, axes = plt.subplots(ncols=2, figsize=(8, 4))
sns.histplot(
    data=gve.df_g_tau_bursts,
    x="Number of bursts\n(per 60 s)",
    hue=constants.TAU_KCC2,
    multiple="stack",
    palette=COLOR.TAU_SM.cmap,
    hue_norm=COLOR.TAU_SM.norm,
    ax=axes[0],
    alpha=1,
    stat="percent",
)
sns.histplot(
    data=gve.df_g_tau_bursts,
    x="Number of bursts\n(per 60 s)",
    hue=constants.G_GABA,
    multiple="stack",
    palette=COLOR.G_GABA_SM.cmap,
    hue_norm=COLOR.G_GABA_SM.norm,
    ax=axes[1],
    alpha=1,
    stat="percent",
)

# most bursts are <= 15 so for visualisation, set this to the norm max
gve.plot(egabas=5, bursts_max=15)
if SAVE_FIGS:
    from style.figure import plot_save

    plot_save(f"output/{gve.fig_name}_supp.jpg", figs=[fig], close=False)
    gve.save_figure(figs=gve.figs, **save_args)

## Supplementary: investigate how excitatory parameters affect the number of bursts.

Specifically, we vary:

1. $g_{AMPA_{max}}$
2. $g_{NMDA_{max}}$

And run static $Cl^-$ simulations at hyperpolarised (blue), close to $V_{rest}$ (purple), and depolarised (red) EGABA values.


In [ ]:
from scripts.lrdfigure_gampa import Params

exc_params = Params(
    gGABAs=[
        # 0,
        25,
        50,
        100,
        # 200,
    ],
    gAMPAs=np.round(np.arange(0, 20.0001, 5.0), 0),
    gNMDAs=[5.0, 7.5, 10.0],
    seeds=(None, 1013, 12987, 1234, 1837),
)
exc_params.run()
exc_params.process()
exc_params.plot(egabas=[-42, -56, -70])

if SAVE_FIGS:
    exc_params.save_figure(**save_args)

# Playground


`None` keeps default values (see `core.sim.py`)


In [ ]:
from core.sim import single_run

net, results = single_run(
    N=None,
    duration=5 * second,
    dt=None,  # Simulation params
    Mg2_t0=None,
    zero_mag_wash_rate=None,  # Seizure params
    benzo_onset_t=None,
    benzo_wash_rate=None,
    benzo_strength=None,
    benzo_off_t=None,  # drug params
    p=None,
    p_ee=None,
    p_ei=None,
    p_ie=None,
    p_ii=None,  # Connection params
    w=None,
    w_ee=None,
    w_ei=None,
    w_ie=None,
    w_ii=None,  # weight params
    U_0=None,
    tau_d=None,
    tau_f=None,  # STP params
    g_AMPA_max=None,
    g_NMDA_max=None,
    g_GABA_max=None,  # conductances
    E_Cl_0=None,
    E_Cl_target=None,
    E_Cl_end=None,
    E_Cl_pop=None,  # ECl params
    length=None,  # neuron params
    dyn_cl=None,
    manual_cl=None,  # dynamic/manual arg
    tau_KCC2_E=None,
    tau_KCC2_I=None,
    num_ecl_steps=None,  # dynamic + manual params
    __build=True,
    __save_run=False,
    __monitors=None,
    nrn_idx_i=None,  # Brian2 args
    run_seed=None,
    __plot=True,
    __device_directory=None,
)